Hi Everyone,<br>
This notebook contains viaualizations based on 911 calls data. The questions and visualizations are part of udemy data capstone project.

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

from subprocess import check_output
print(check_output(["ls", "../input"]).decode("utf8"))

# Any results you write to the current directory are saved as output.

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
plt.style.use(u'ggplot')

In [ ]:
df = pd.read_csv('../input/911.csv')

In [ ]:
df.info()

In [ ]:
df.head()

**Basic Questions**

**What are the top 5 zipcodes for 911 calls?**

In [ ]:
df['zip'].value_counts().head(5)

**What are the top 5 townships(twp) for 911 calls?**

In [ ]:
df['twp'].value_counts().head(5)

**Take a look at the 'title' column, how many unique title codes are there?**

In [ ]:
df['title'].nunique()

**Creating new features**

In the titles column, there are "Reasons/Departments" specified before the title code. These are EMS, Fire, and Traffic. Use .apply() with a custom lambda expression to create a new column called "Reason" that contains this string value.

For Example, if the column value is EMS:BACK PAIN/INJURY, the reasonn column value would be EMS.

In [ ]:
df['reasons']=df['title'].apply(lambda x:x.split(':')[0])

**What is the most common reason for a 911 call based off this new column?**

In [ ]:
df['reasons'].value_counts().head(3)

**Now use Seaborn to create a countplot of 911 calls by Reason.**

In [ ]:
sns.countplot(x='reasons',data=df)

**Now let us begin to focus on time information. What is the data type of the objects in timeStamp column?**

In [ ]:
df['timeStamp'].dtype

**These timestamps are of object type. Use pd.to_datetime to convert the column from strings to DataTime objects.**

In [ ]:
df['timeStamp'] = pd.to_datetime(df['timeStamp'])

**We can grab specific attributes from a Datetime object by calling them. For example:**

In [ ]:
time = df['timeStamp'].iloc[0]
time.hour

**Use .apply() to create 3 new columns called Hour, Month, and Day of Week. You will create these columns based off of the timeStamp column.**

In [ ]:
df['hour'] = df['timeStamp'].apply(lambda x : x.hour)

In [ ]:
df['month'] = df['timeStamp'].apply(lambda x : x.month)
df['day of week'] = df['timeStamp'].dt.dayofweek
df.head()

**Use .map() with this dictionary to map the actual string names to the day of the week:**

In [ ]:
dmap = {0:'Mon',1:'Tue',2:'Wed',3:'Thu',4:'Fri',5:'Sat',6:'Sun'}
df['day of week'] = df['day of week'].map(dmap)

Now use seaborn to create a countplot of the Day of Week column with te hue based off of the reaosn column.

In [ ]:
plt.figure(figsize=(10,6))
sns.countplot('day of week',data=df,hue='reasons',palette='Set1')
plt.legend(bbox_to_anchor=(1, 1), loc=2)

Now do the same for month

In [ ]:
plt.figure(figsize=(10,6))
sns.countplot(x='month',data=df,hue='reasons',palette='Set1')
plt.legend(bbox_to_anchor=(1, 1), loc=2)

In [ ]:
byMonth = df.groupby(by=df['month']).count()

In [ ]:
byMonth.head()

In [ ]:
df['month'].groupby(by=df['month']).count().plot()

Use Seaborn's lmplot() to create a linear fit on the number of calls per month.

In [ ]:
# create a new column in byMonth dataframe
byMonth['count'] = df['month'].value_counts()
byMonth['month'] = byMonth.index
# reset index from month to something else keeping previous index as a column
byMonth.reset_index(drop=True)

In [ ]:
sns.lmplot(x='month',y='count',data=byMonth,fit_reg=True)

Create a new column called 'date' that contains the date from the timeStamp column.

In [ ]:
df['date'] = df['timeStamp'].apply(lambda x :  x.date())

Now Groupby this date column with the count() aggregate and create a plot of counts of 911 calls.

In [ ]:
plt.figure(figsize=(12,4))
df['month'].groupby(by=df['date'],sort=True).count().plot()

Recreate the above plot but create 3 separate plots with each plot representing a Reason for 911 call.

In [ ]:
plt.figure(figsize=(12,5))
df[df['reasons']=='Traffic']['month'].groupby(by=df['date']).count().plot()
plt.title('Traffic')
plt.ylim(0,600)

In [ ]:
plt.figure(figsize=(12,5))
df[df['reasons']=='Fire']['month'].groupby(by=df['date']).count().plot()
plt.title('Fire')
plt.ylim(0,180)

In [ ]:
plt.figure(figsize=(12,4))
df[df['reasons']=='EMS']['month'].groupby(by=df['date']).count().plot()
plt.title('EMS')
plt.ylim(0,250)

To create heatmaps, we need to restructure the dataframe so that the columns become the Hours and the Index becomes the Day of the Week. 

In [ ]:
df2 = df[['day of week','hour']]
df2 = df2.groupby(['day of week','hour']).size()
df2 = df2.unstack(level=-1)

In [ ]:
df2

Now create a heatmap using this new DataFrame

In [ ]:
plt.figure(figsize=(12,6))
sns.heatmap(data=df2,vmin=300,vmax=3000,cmap='viridis',cbar=True)

Now create a clustermap using this dataframe

In [ ]:
sns.clustermap(data=df2,figsize=(15,10))

Now repeat thses same plots and operations for a dataframe that shows month as the column.

In [ ]:
df3 = df[['day of week','month']]
df3 = df3.groupby(['day of week','month']).size()
df3 = df3.unstack(level=-1)
df3

In [ ]:
plt.figure(figsize=(12,6))
sns.heatmap(data=df3,cmap='viridis',vmin=1000,vmax=5000)

In [ ]:
sns.clustermap(data=df3,figsize=(15,10))

#### Thank You!!!